In [2]:
import gurobipy as gp
from gurobipy import GRB
import pickle
from pathlib import Path

In [3]:
folder = Path("instances")
instance = "generic_5_50.pkl"

instance_path = folder / instance

with instance_path.open("rb") as file:
    problem_instance = pickle.load(file)

project_info, student_info  = problem_instance

In [4]:
print(project_info)

                                 name  desired#groups  max#groups  \
0                  Financial Planning               3           4   
1                    Event Management               3           6   
2  Broadcasting, Radio, TV Management               3           4   
3                   Digital Marketing               4           7   
4                   People Management               4           7   

   ideal_group_size  min_group_size  max_group_size  pen_groups  pen_size  
0                 3               2               4           3         2  
1                 3               2               4           2         2  
2                 2               1               3           1         3  
3                 3               2               6           3         1  
4                 3               2               4           1         2  


In [5]:
print(student_info)

                   name  fav_partners    project_prefs
0            Bobby Hill  [49, 38, 26]  (0, 2, 1, 1, 3)
1            Liam Smith   [26, 44, 6]  (2, 0, 3, 3, 1)
2          Walter Jones  [27, 47, 43]  (0, 3, 0, 1, 2)
3        Jessica Hughes  [42, 14, 10]  (1, 1, 2, 1, 3)
4            Joe Morris    [3, 1, 46]  (1, 1, 2, 2, 2)
5       Ashley Martinez   [3, 22, 30]  (1, 1, 2, 1, 3)
6           Brenda Hill   [1, 19, 34]  (1, 0, 3, 3, 1)
7          Janice Adams    [47, 3, 6]  (1, 1, 2, 2, 1)
8     Christina Jimenez   [4, 31, 39]  (2, 1, 2, 1, 2)
9         Steven Rivera  [37, 31, 11]  (1, 3, 1, 3, 0)
10         Mark Sanders  [42, 36, 38]  (3, 2, 3, 0, 3)
11       Kimberly Perez    [9, 4, 43]  (2, 1, 1, 2, 1)
12        Carl Castillo  [25, 39, 19]  (2, 3, 3, 2, 3)
13        Carolyn Kelly  [16, 17, 15]  (3, 3, 2, 3, 0)
14          Jordan Ward    [3, 8, 38]  (2, 1, 2, 2, 2)
15    Madison Rodriguez   [9, 38, 43]  (1, 2, 1, 3, 1)
16      Matthew Roberts   [13, 1, 20]  (3, 1, 2, 2, 0)
17        

In [6]:
reward_bilateral = 2
penalty_unassignment = 3

In [7]:

m = gp.Model()

Set parameter Username
Set parameter LicenseID to value 2654427
Academic license - for non-commercial use only - expires 2026-04-19


In [8]:
project_ids = range(len(project_info))
student_ids = range(len(student_info))

projects_group_ids = {
    project_id: range(project_info["max#groups"][project_id])
    for project_id in project_ids
}


In [9]:
x = m.addVars(
    (
        (project_id, group_id, student_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
        for student_id in student_ids
    ),
    vtype=GRB.BINARY,
    name="assign",
)
m.update()

In [10]:
y = m.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    vtype=GRB.BINARY,
    name="establish_group",
)

In [11]:
favorite_partners_students = student_info["fav_partners"].tolist()
mutual_pairs = set()

for student_id, favorite_partners in enumerate(favorite_partners_students):
    for partner_id in favorite_partners:
        if partner_id <= student_id:
            continue
        if student_id in favorite_partners_students[partner_id]:
            mutual_pairs.add((student_id, partner_id))

In [12]:
z = m.addVars(mutual_pairs, vtype=GRB.BINARY, name="not_realize_bilateral_cooperation_wish")

In [13]:
v = m.addVars(student_ids, name = "student_unassigned")

In [14]:
gs_surplus = m.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    name="group_size_surplus",
)

In [15]:
gs_deficit = m.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    name="group_size_deficit",
)

In [16]:
project_preferences = {
    (student_id, project_id): student_info["project_prefs"][student_id][project_id]
    for student_id in student_ids
    for project_id in project_ids
}

In [17]:
rewards_student_project_preference = gp.quicksum(
    project_preferences[student_id, project_id] * x[project_id, group_id, student_id]
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
    for student_id in student_ids
)

rewards_bilateral_cooperation_wish_realized = reward_bilateral * gp.quicksum(
    1 - z[*mutual_pair] for mutual_pair in mutual_pairs
)

penalties_student_unassigned = penalty_unassignment * gp.quicksum(v.values())

penalties_num_of_groups_exceeding_num_offered_by_project = gp.quicksum(
    project_info["pen_groups"][project_id] * y[project_id, group_id]
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
    if group_id >= project_info["desired#groups"][project_id]
)

penalties_deviation_from_ideal_group_size_of_project = gp.quicksum(
    project_info["pen_size"][project_id]
    * (gs_surplus[project_id, group_id] + gs_deficit[project_id, group_id])
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
)

m.setObjective(
    rewards_student_project_preference
    + rewards_bilateral_cooperation_wish_realized
    - penalties_student_unassigned
    - penalties_num_of_groups_exceeding_num_offered_by_project
    - penalties_deviation_from_ideal_group_size_of_project,
    sense=GRB.MAXIMIZE,
)

In [18]:
m.addConstrs(
    (x.sum("*", "*", student_id) + v[student_id] == 1 for student_id in student_ids),
    name="penalty_if_unassigned",
)
m.update()

In [19]:
m.addConstrs(
    (
        y[project_id, group_id] <= y[project_id, group_id - 1]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
        if group_id > 0
    ), name="only_consecutive_group_ids"
)
m.update()

In [20]:
m.addConstrs(
    (
        x.sum(project_id, group_id, "*")
        >= project_info["min_group_size"][project_id] * y[project_id, group_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_min_group_size"
)
m.update()

In [21]:
m.addConstrs(
    (
        x.sum(project_id, group_id, "*")
        <= project_info["max_group_size"][project_id] * y[project_id, group_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="cap_group_size_at_max"
)
m.update()

In [22]:
m.addConstrs(
    (
        gs_surplus[project_id, group_id]
        >= x.sum(project_id, group_id, "*")
        - project_info["ideal_group_size"][project_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_correct_group_size_surplus"
)
m.update()

In [23]:
m.addConstrs(
    (
        gs_deficit[project_id, group_id]
        >= project_info["ideal_group_size"][project_id]
        - x.sum(project_id, group_id, "*")
        - project_info["max_group_size"][project_id] * (1 - y[project_id, group_id])
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_correct_group_size_deficit"
)
m.update()

In [24]:
max_num_groups = max(project_info["max#groups"])


unique_group_identifiers = {
    (project_id, group_id): project_id + group_id / max_num_groups
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
}

In [25]:
num_projects = len(project_info)

m.addConstrs(
    (
        z[first_student_id, second_student_id] * num_projects
        >= sum(
            unique_group_identifiers[project_id, group_id]
            * (
                x[project_id, group_id, first_student_id]
                - x[project_id, group_id, second_student_id]
            )
            for project_id in project_ids
            for group_id in projects_group_ids[project_id]
        )
        for (first_student_id, second_student_id) in mutual_pairs
    ), name="ensure_correct_inidicator_different_group_1"
)
m.update()

In [26]:
m.addConstrs(
    (
        z[first_student_id, second_student_id] * num_projects
        >= sum(
            unique_group_identifiers[project_id, group_id]
            * (
                x[project_id, group_id, second_student_id]
                - x[project_id, group_id, first_student_id]
            )
            for project_id in project_ids
            for group_id in projects_group_ids[project_id]
        )
        for (first_student_id, second_student_id) in mutual_pairs
    ), name="ensure_correct_inidicator_different_group_2"
)
m.update()

In [27]:
m.Params.TimeLimit = 60
m.optimize()

Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 251 rows, 1567 columns and 10866 nonzeros
Model fingerprint: 0x3e56e929
Variable types: 106 continuous, 1461 integer (1461 binary)
Coefficient statistics:
  Matrix range     [1e-01, 6e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective -84.0000000
Presolve added 28 rows and 28 columns
Presolve time: 0.00s
Presolved: 279 rows, 1595 columns, 6806 nonzeros
Variable types: 0 continuous, 1595 integer (1532 binary)

Root relaxation: objective 1.831247e+02, 777 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |  

In [28]:
for project_id in project_ids:
    print("\n")
    print(project_info["name"][project_id])
    for group_id in projects_group_ids[project_id]:
        print("\nGroup", group_id + 1)
        for student_id in student_ids:
            if x[project_id, group_id, student_id].X > 0.5:
                print(student_info["name"][student_id], student_id)



Financial Planning

Group 1
Amber Campbell 21
Barbara Cook 25
Brandon Brooks 33

Group 2
Christina Jimenez 8
Maria Chavez 24
Kyle Ramos 31

Group 3
Carolyn Kelly 13
Matthew Roberts 16
John Anderson 17
Amber Roberts 20

Group 4


Event Management

Group 1
Bobby Hill 0
Hannah King 38
Raymond Johnson 49

Group 2
Jennifer Richardson 34
Mason Nelson 42
Jacob Price 45

Group 3
Walter Jones 2
Madison Rodriguez 15
Liam Perez 43

Group 4

Group 5

Group 6


Broadcasting, Radio, TV Management

Group 1
Carl Castillo 12
Judith Rivera 39

Group 2
Mark Sanders 10
Sara Miller 36

Group 3

Group 4


Digital Marketing

Group 1
Angela Reed 19
Elizabeth Peterson 28
Dennis Myers 41

Group 2
Liam Smith 1
Brenda Hill 6
Mark Johnson 44

Group 3
Steven Rivera 9
Kimberly Perez 11
Janice Morales 18

Group 4
Janice Adams 7
Joshua Clark 40
Russell Jackson 47

Group 5

Group 6

Group 7


People Management

Group 1
Jessica Hughes 3
Jordan Ward 14
Jennifer Wilson 26

Group 2
Ashley Martinez 5
Christina Thompson 22

In [29]:
# m.write("SSPAGDP.lp")

In [30]:
m.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment
